In [1]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [6]:
BASE_DIR = '../scripts/data'
IMAGE_DIR2 = '../../scripts/PH_IMAGES_DIR'
RANDOM_SEED = 7 # for reproducibility

# these relate to training the CNN to predict nightlights
CNN_TRAIN_IMAGE_DIR = BASE_DIR
CNN_SAVE_DIR = os.path.join(BASE_DIR, 'models')
os.makedirs(CNN_SAVE_DIR, exist_ok=True)

# Preprocess

In [7]:
df_download = pd.read_csv('../data/image_download_locs.csv')

In [8]:
df_download.head()

,Unnamed: 0,lon,lat,emission,filename,nightlights,nightlights_bin
0,0,121.087500,18.612500,7.467103,18.6125_121.0875.jpg,0.577604,1
1,1,120.779167,18.604167,6.376585,18.604166666666664_120.77916666666668.jpg,0.566483,1
2,2,121.054167,18.604167,7.586672,18.604166666666664_121.05416666666667.jpg,0.577604,1
3,3,121.062500,18.604167,8.018453,18.604166666666664_121.0625.jpg,0.577604,1
4,4,121.070833,18.604167,7.548463,18.604166666666664_121.07083333333334.jpg,0.577604,1


In [9]:
# the distribution
(df_download['nightlights_bin']==1).mean(), (df_download['nightlights_bin']==2).mean(), (df_download['nightlights_bin']==0).mean()


(0.329743320324227, 0.6055488842189533, 0.00020014009806864806)

Creates train/valid folders

The model will use these directly.

In [10]:
# we need this renaming because images can have the same name when they belong to different clusters
# this makes unique image names, even though some are really the same image
#we probably dpont need this rite - kirsten 
#df_download['image_renamed'] = df_download['lat'].astype(str) + '_' + df_download['lon'].astype(str) + '_' + df_download['filename']


In [11]:
df_download.head()

,Unnamed: 0,lon,lat,emission,filename,nightlights,nightlights_bin
0,0,121.087500,18.612500,7.467103,18.6125_121.0875.jpg,0.577604,1
1,1,120.779167,18.604167,6.376585,18.604166666666664_120.77916666666668.jpg,0.566483,1
2,2,121.054167,18.604167,7.586672,18.604166666666664_121.05416666666667.jpg,0.577604,1
3,3,121.062500,18.604167,8.018453,18.604166666666664_121.0625.jpg,0.577604,1
4,4,121.070833,18.604167,7.548463,18.604166666666664_121.07083333333334.jpg,0.577604,1


In [12]:
pic_list = df_download['filename'].values.tolist()
to_pick = int(0.8*len(pic_list)); to_pick

63955

In [13]:
np.random.seed(RANDOM_SEED)
inds = np.arange(len(pic_list))
train_ind = np.random.choice(np.arange(len(pic_list)), to_pick, replace=False)
valid_ind = np.delete(inds, train_ind)

In [14]:
pic_list = np.array(pic_list)
train_im = pic_list[train_ind]
valid_im = pic_list[valid_ind]

In [15]:
os.makedirs(os.path.join(IMAGE_DIR2, 'train'), exist_ok=False)
os.makedirs(os.path.join(IMAGE_DIR2, 'valid'), exist_ok=False)

labels = ['1', '2', '0']
for l in labels:
    os.makedirs(os.path.join(IMAGE_DIR2, 'train', l), exist_ok=False)
    os.makedirs(os.path.join(IMAGE_DIR2, 'valid', l), exist_ok=False)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: '../../scripts/PH_IMAGES_DIR\\train'

In [17]:
t = df_download.iloc[train_ind]
v = df_download.iloc[valid_ind]

In [18]:
t

,Unnamed: 0,lon,lat,emission,filename,nightlights,nightlights_bin
9951,9951,121.029167,15.595833,9.021405,15.595833333333331_121.02916666666668.jpg,8.273617,2
77792,77792,124.912500,6.545833,7.900402,6.5458333333333325_124.9125.jpg,4.789633,1
73857,73857,124.895833,7.120833,10.672616,7.120833333333333_124.89583333333334.jpg,5.295788,2
63199,63199,124.479167,8.562500,17.310165,8.562499999999998_124.47916666666669.jpg,12.109493,2
7091,7091,120.512500,15.920833,11.081390,15.920833333333333_120.5125.jpg,12.023355,2
...,...,...,...,...,...,...,...
16392,16392,120.154167,14.954167,13.985948,14.954166666666666_120.15416666666668.jpg,12.683646,2
55189,55189,122.612500,9.862500,11.582243,9.8625_122.6125.jpg,4.847798,1
63800,63800,123.304167,8.495833,8.928985,8.495833333333332_123.30416666666667.jpg,4.885919,1
9952,9952,121.037500,15.595833,8.783023,15.595833333333331_121.0375.jpg,8.816211,2


In [19]:
len(t), len(v)

(63955, 15989)

In [20]:
# copies images to their respective folders
# YES this does mean an image can be in train and valid, but that's ok
# because we do not really care about accuracy on nightlights
print('copying train images')
for im_name, im_renamed, l in tqdm(zip(t['filename'], t['filename'], t['nightlights_bin']), total=len(t)):
    shutil.copy(os.path.join(IMAGE_DIR2, im_name), os.path.join(IMAGE_DIR2, 'train', str(l), im_renamed))

    #shutil.copy(os.path.join(IMAGE_DIR2, im_name), os.path.join(IMAGE_DIR2, 'train', str(l), im_renamed))
    # i removed the str(l) bec we only have one image per cluster/point anyways -kirsten 
    
print('copying valid images')
for im_name, im_renamed, l in tqdm(zip(v['filename'], v['filename'], v['nightlights_bin']), total=len(v)):
    shutil.copy(os.path.join(IMAGE_DIR2, im_name), os.path.join(IMAGE_DIR2, 'valid', str(l), im_renamed))

copying train images


  0%|          | 0/63955 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '../../scripts/PH_IMAGES_DIR\\train\\3\\14.695833333333333_121.1375.jpg'

In [21]:
# shows count distribution in train folder, make sure this matches above
counts = []
for l in ['1', '2', '0']:
    counts.append(len(os.listdir(os.path.join(IMAGE_DIR2, 'train', l))))
print(counts)
print([c/sum(counts) for c in counts])
print(sum(counts))

[46053, 1554, 16348]
[0.7200844343679149, 0.02429833476663279, 0.2556172308654523]
63955


In [23]:
# shows count distribution in valid folder
counts = []
for l in ['1', '2', '0']:
    counts.append(len(os.listdir(os.path.join(IMAGE_DIR2, 'valid', l))))
print(counts)
print([c/sum(counts) for c in counts])
print(sum(counts))

[11543, 400, 4046]
[0.7219338295077866, 0.025017199324535618, 0.25304897116767777]
15989


# Train Model
Heavily adapted from the PyTorch CNN training tutorial.

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [25]:
# Top level data directory.

data_dir = IMAGE_DIR2
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "vgg"

# Number of classes in the dataset
num_classes = 3

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 5

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [26]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [27]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
model_ft

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [28]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

Initializing Datasets and Dataloaders...
device: cpu


In [29]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 63955
     Root Location: ../scripts/PH_IMAGES_DIR\train
     Transforms (if any): Compose(
                              RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                              RandomHorizontalFlip(p=0.5)
                              ToTensor()
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                          )
     Target Transforms (if any): None,
 'valid': Dataset ImageFolder
     Number of datapoints: 15989
     Root Location: ../scripts/PH_IMAGES_DIR\valid
     Transforms (if any): Compose(
                              Resize(size=224, interpolation=PIL.Image.BILINEAR)
                              CenterCrop(size=(224, 224))
                              ToTensor()
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                          

In [30]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=1e-4, momentum=0.9)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [31]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [32]:
###### Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/4
----------


  0%|          | 0/7995 [00:00<?, ?it/s]

train Loss: 0.5824 Acc: 0.7290


  0%|          | 0/1999 [00:00<?, ?it/s]

valid Loss: 0.5246 Acc: 0.7523

Epoch 1/4
----------


...

valid Loss: 0.5111 Acc: 0.7603

Epoch 4/4
----------


  0%|          | 0/7995 [00:00<?, ?it/s]

train Loss: 0.5509 Acc: 0.7378


  0%|          | 0/1999 [00:00<?, ?it/s]

valid Loss: 0.5057 Acc: 0.7616

Training complete in 630m 31s
Best val Acc: 0.764776


In [34]:
path = os.path.join(CNN_SAVE_DIR, 'trained_model.pt')
assert not os.path.isfile(path), print('A model is already saved at this location')
print(f'Saving model to {path}')
torch.save(model_ft, path)

Saving model to ../scripts/data\models\trained_model.pt


In [35]:
# lol added this muna
####### wait idk where this came from or who wrote this here - kirfsten
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = torch.load(r"C:\Users\PSHSWVC-CID\Documents\estimating-co2-ph\scripts\data\models\trained_model.pt", map_location=torch.device('cpu')).to(device)

In [36]:
# you can run below if you want to see the final accuracy on nightlights over the train set
model_ft.eval()   # Set model to evaluate mode

criterion = nn.CrossEntropyLoss()
running_loss = 0.0
running_corrects = 0
total = 0


# Iterate over data.
for inputs, labels in tqdm(dataloaders_dict['train']):
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)

    # statistics
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    
    total += len(preds)
        
print(running_corrects.double()/total)

  0%|          | 0/7995 [00:00<?, ?it/s]

tensor(0.7603, dtype=torch.float64)
